实现多层感知机模型

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)
import tensorflow as tf
sess=tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
in_units=784            #输入节点数
h1_units=300            #隐含层的输出节点数
#隐含层参数设置（偏置赋值为0，并将权重初始化为截断的正态分布，其标准差为0.1,从而避免0梯度）
w1=tf.Variable(tf.truncated_normal([in_units,h1_units],stddev=0.1))
b1=tf.Variable(tf.zeros([h1_units]))
#Softmax输出层参数设置
w2=tf.Variable(tf.zeros([h1_units,10]))
b2=tf.Variable(tf.zeros([10]))

#定义输入x的placeholder
x=tf.placeholder(tf.float32,[None,in_units])
#定义输出
y_=tf.placeholder(tf.float32,[None,10])
#在训练和预测时，Dropout的比率keep_prob是不一样的，通常在训练时小于1，而预测时大于1，所以Dropout的比率也作为输入
keep_prob=tf.placeholder(tf.float32)

#定义模型的结构
#首先定义隐含层(激活函数为ReLU)
hidden1=tf.nn.relu(tf.matmul(x,w1)+b1) 
#调用tf.nn.dropout实现Dropout的功能
#（dropout: 在训练时keep_prob是小于1的，随机的将一部分节点置为0，用以制作随机性，防止过拟合，在预测时应该等于1，使用全部的特征来预测样本的类别）
hidden1_drop=tf.nn.dropout(hidden1,keep_prob)
#定义输出层，使用Softmax作为激活函数
y=tf.nn.softmax(tf.matmul(hidden1_drop,w2)+b2)


#定义损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y)))

#定义优化算法，选择自适应的Adagrad
train_step=tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

#初始化全局参数(加入keep_prob作为计算图的输入，并且在训练的时候设置为0.75,即保留75%的节点，其余25%的设置为0)
tf.global_variables_initializer().run()
for i in range(3000):
    batch_xs,batch_ys=mnist.train.next_batch(100)
    train_step.run({x:batch_xs,y_:batch_ys,keep_prob:0.75})
#对模型进行准确率的评估
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

0.098
